# Visualizing a PPI

In [ ]:
import os
import datetime
import numpy as np

from netCDF4 import Dataset

import chart

In [ ]:
file =  os.path.expanduser('~/Downloads/PX-20170220-050706-E2.4-Z.nc')

with Dataset(file, mode='r') as nc:
    elev = np.array(nc.variables['Elevation'][:], dtype=np.float32)
    azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
    gatewidth = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
    values = np.array(nc.variables['Corrected_Intensity'][:], dtype=np.float32)
    longitude = nc.getncattr('Longitude')
    latitude = nc.getncattr('Latitude')
    sweepElev = nc.getncattr('Elevation')
    sweepTime = nc.getncattr('Time')

## Overlay for the Plot

In [ ]:
overlay = chart.atlas.Overlay((longitude, latitude))

In [ ]:
radii = np.concatenate(([1.0], np.arange(10.0, 61.0, 10.0)))
overlay.setRingRadii(radii)
overlay.load()

In [ ]:
r = 1.0e-3 * np.arange(values.shape[1]) * gatewidth[0]
a = np.deg2rad(azim)
t = datetime.datetime.utcfromtimestamp(sweepTime)
timestr = t.strftime('%Y/%m/%d %H:%M:%S')
title = f'{timestr} UTC  EL: {elevation:.2f}°'
symbol = 'Z'

In [ ]:
ppi = chart.chart.Image(a, r, values, style=symbol, overlay=overlay, title=title, figsize=(800, 640), maxrange=50.0)

In [ ]:
ppi.image()